In [1]:
import sqlite3
import pandas as pd
import import_ipynb
from analysisMagic import dfm3

<string>:1: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,33,34,36,38,40,41,42,48,53,54,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   setName      864 non-null    object
 1   setCode      864 non-null    object
 2   releaseDate  864 non-null    object
 3   releaseYear  864 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 27.1+ KB


In [2]:
# creates db if it doesn't already exist
conn = sqlite3.connect("../sql/magic.db")

In [3]:
dfm3.head()

,name,setCode,setName,language,types,colors,rarity,cardFinish,releaseDate,releaseYear,gameAvailability,priceProvider,price,avgMarketPrice,currency,providerListing,date,uuid
0,Air Elemental,LEA,Limited Edition Alpha,English,Creature,U,uncommon,normal,1993-08-05,1993,paper,tcgplayer,119.66,160.26,USD,retail,2025-08-26,27e92f54-0084-57c2-85e5-197e026fab5c
1,Air Elemental,LEA,Limited Edition Alpha,English,Creature,U,uncommon,normal,1993-08-05,1993,paper,cardkingdom,229.99,160.26,USD,retail,2025-08-26,27e92f54-0084-57c2-85e5-197e026fab5c
2,Air Elemental,LEA,Limited Edition Alpha,English,Creature,U,uncommon,normal,1993-08-05,1993,paper,cardsphere,131.12,160.26,USD,retail,2025-08-26,27e92f54-0084-57c2-85e5-197e026fab5c
3,Ancestral Recall,LEA,Limited Edition Alpha,English,Instant,U,rare,normal,1993-08-05,1993,paper,cardsphere,20460.00,20460.00,USD,retail,2025-08-26,1c17ce18-bf3e-558b-9389-632588f93851
4,Animate Artifact,LEA,Limited Edition Alpha,English,Enchantment,U,uncommon,normal,1993-08-05,1993,paper,tcgplayer,61.05,55.07,USD,retail,2025-08-26,e035e37e-cb8e-5f12-a5db-fe7f927a3457


In [4]:
cardsDf = dfm3[['name', 'setName', 'setCode', 'releaseDate', 'language', 'types', 'colors', 'rarity', 'gameAvailability', 'uuid']]
pricesDf = dfm3[['date', 'cardFinish', 'price', 'priceProvider', 'avgMarketPrice', 'providerListing', 'uuid']]

cardsDf.to_sql('cards', conn, index=False, if_exists='replace')
pricesDf.to_sql('prices', conn, index=False, if_exists='replace')
conn.commit()

In [5]:
# cardsDf = dfm4[['name', 'setName', 'setCode', 'releaseDate', 'language', 'types', 'colors', 'rarity', 'gameAvailability', 'uuid']]
# pricesDf = dfm4[['date', 'cardFinish', 'avgMarketPrice', 'providerListing', 'uuid']]

# cardsDf.to_sql('cards', conn, index=False, if_exists='replace')
# pricesDf.to_sql('prices', conn, index=False, if_exists='replace')
# conn.commit()

In [6]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,cards
1,prices


### How many unique cards are printed per set (top ten)?

In [7]:
query1 = """
SELECT
    c.setname,
    COUNT(DISTINCT c.uuid) as numCards,
    strftime('%Y', c.releaseDate) AS releaseYear
FROM cards c
GROUP BY c.setname
ORDER BY numCards DESC
Limit 10;
"""

result1 = pd.read_sql(query1, conn)
result1

,setName,numCards,releaseYear
0,The List,5028,2020
1,Secret Lair Drop,1888,2019
2,Universes Beyond: Doctor Who,1166,2023
3,Fallout,1056,2024
4,Commander Masters,1036,2023
5,Commander Legends: Battle for Baldur's Gate,952,2022
6,Lord of the Rings: Tales of Middle-Earth,823,2023
7,Jump Start 2022,820,2022
8,Foundations Jumpstart,764,2024
9,Commander Legends,712,2020


### What is the best price vs. market price for a particular card?

In [15]:
query2 = """
SELECT DISTINCT
    c.name,
    c.setname,
    p.priceProvider,
    p.price,
    p.avgMarketPrice,
    p.cardFinish,
    c.uuid
FROM cards c
JOIN prices p ON c.uuid = p.uuid
WHERE c.name = 'Presence of the Master'
ORDER BY p.price DESC;
"""

result2 = pd.read_sql(query2, conn)
result2

,name,setName,priceProvider,price,avgMarketPrice,cardFinish,uuid
0,Presence of the Master,Legends,tcgplayer,10.75,10.42,normal,3c540848-d4ca-5441-a098-51a295e39aef
1,Presence of the Master,Legends,cardsphere,10.52,10.42,normal,3c540848-d4ca-5441-a098-51a295e39aef
2,Presence of the Master,Legends,cardkingdom,9.99,10.42,normal,3c540848-d4ca-5441-a098-51a295e39aef
3,Presence of the Master,Urza's Saga,cardkingdom,0.69,0.52,normal,cd9ed8e9-3778-5e5c-907e-db5f41dbc215
4,Presence of the Master,Urza's Saga,cardsphere,0.44,0.52,normal,cd9ed8e9-3778-5e5c-907e-db5f41dbc215
5,Presence of the Master,Urza's Saga,tcgplayer,0.43,0.52,normal,cd9ed8e9-3778-5e5c-907e-db5f41dbc215
